In [2]:
%load_ext autoreload
%autoreload 2

In [22]:
from import_casa import casa
from casa import Cadet

In [18]:
import pickle
import numpy as np
import pandas as pd

## Sample opinions (every 20)

In [68]:
with (casa.get_data_path() / f"threads/cht2021-JanMay-op-every20.pkl").open("rb") as fin:
    op_sample = pickle.load(fin)

In [20]:
op_sample[3]

<Opinion [] 小小比: 5G再過1、2年才會普遍，，目前不要急著...>

In [70]:
cadet = Cadet.load(casa.get_data_path()/"cadet/op20.2")
cadet

[INFO] 2021-06-29 17:16:16,045 gensim.utils: loading KeyedVectors object from E:\LangOn\casa\data\cadet\op20.2\ft-2020.kv
[INFO] 2021-06-29 17:16:16,328 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-06-29 17:16:16,328 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-06-29 17:16:16,329 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-06-29 17:16:16,329 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-06-29 17:16:16,445 gensim.utils: FastTextKeyedVectors lifecycle event {'fname': 'E:\\LangOn\\casa\\data\\cadet\\op20.2\\ft-2020.kv', 'datetime': '2021-06-29T17:16:16.445979', 'gensim': '4.0.0', 'python': '3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'loaded'}


<Cadet: FastText(9851, 200), Seeds(85)>

In [86]:
from tqdm.auto import tqdm
srv_list = list(cadet.lexicon.get_services().keys())
ent_list = list(cadet.lexicon.get_entities().keys())
cadet_failed_ids = []
for op_x in tqdm(op_sample):
    try:
        res = cadet.detect(op_x.text, summary=False)        
    except:
        cadet_failed_ids.append(op_x.id)
        continue
    ent_probs = res["entity_probs"]
    srv_probs = res["service_probs"]
    top_ent = np.argmax(ent_probs)
    top_srv = np.argmax(srv_probs)
    ent_maxp = np.max(ent_probs)
    srv_maxp = np.max(srv_probs)
    setattr(op_x, "cadet_entity", ent_list[top_ent] if ent_maxp > 0.3 else None)
    setattr(op_x, "cadet_service", srv_list[top_srv] if srv_maxp > 0.1 else None)
    setattr(op_x, "ent_probs", ent_probs)
    setattr(op_x, "srv_probs", srv_probs)
    setattr(op_x, "tok_logits", np.random.randn(min(len(op_x.text), 500), 3))
    setattr(op_x, "pred_polarity", np.random.randint(3))

In [77]:
len(cadet_failed_ids)

22

In [78]:
from collections import Counter
ent_counter = Counter(map(lambda x: getattr(x, "cadet_entity", None), op_sample))

In [79]:
ent_counter.most_common()

[(None, 3475),
 ('中華電信', 540),
 ('遠傳電信', 497),
 ('台灣大哥大', 379),
 ('亞太電信', 255),
 ('台灣之星', 243),
 ('無框行動', 16)]

In [49]:
from itertools import islice
xx = list(islice(filter(lambda x: getattr(x, "cadet_entity", None) == "台灣大哥大", op_sample), 5))

In [64]:
xx[0].text

'如題，最近一次更新之後，大概每三把就有一把一按開始對戰就卡在下面這個畫面，會不會太扯。你說紅wifi很卡就算了，這樣直接被算掛機欸，變成電腦打之後不小心贏了還不會加盃。最近打到有點心累，20000盃的帳號好不容易要可以全部滿級了，結果現在這種情況根本沒辦法爬盃，哭啊。(再抱怨一下，20000盃到現在還沒有一隻傳奇角哦啾咪）'

In [71]:
cadet.detect(xx[0].text)

{'entity': ['中華電信', '台灣大哥大', '遠傳電信', '台灣之星', '亞太電信', '無框行動'],
 'entity_probs': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667]),
 'service': [('加值服務', 'vowifi'),
  ('通訊品質', '涵蓋'),
  ('加值服務', '國際漫遊'),
  ('加值服務', 'esim'),
  ('加值服務', '行動支付')],
 'service_probs': array([0.05904923, 0.05410636, 0.05338702, 0.04729032, 0.04625884]),
 'seeds': ['VoLTE', '山區', '出國', '拆台', 'esim'],
 'seed_probs': array([0.02191027, 0.02007621, 0.0198093 , 0.01851987, 0.01754712])}

In [87]:
with (casa.get_data_path() / f"threads/cht2021-JanMay-op-every20-attr.pkl").open("wb") as fout:
    pickle.dump(op_sample, fout)